In [1]:
from web3 import Web3
import json
web3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
assert web3.isConnected()

In [2]:
with open ("contract/abi.json") as f:
    contract_abi = f.read()
with open("config.json") as f:
    config = json.load(f)


In [3]:
contract_address = config["contract_address"]
private_key = config["private_key1"]

In [4]:
contract = web3.eth.contract(address=contract_address, abi=contract_abi)
account = web3.eth.account.from_key(private_key)
web3.eth.default_account = account.address
print("address : %s" % account.address)
nonce = web3.eth.getTransactionCount(account.address)

address : 0xa508dD875f10C33C52a8abb20E16fc68E981F186


In [5]:
tx_params = {
    'nonce': nonce,
    'gasPrice': web3.eth.gas_price,
    'gas': 5000000, 
}

In [6]:
new_project_tx = contract.functions.new_project("project1").buildTransaction(tx_params)
signed_tx = web3.eth.account.sign_transaction(new_project_tx, private_key)
try:
    tx_hash = web3.eth.send_raw_transaction(signed_tx.rawTransaction)
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print ("tx_receipt : %s" % tx_receipt)
except Exception as e:
    print (str(e).split("\'")[3])

tx_receipt : AttributeDict({'transactionHash': HexBytes('0xb71410aa3eda3b8cb96a2ffd14a9f4d5ed710975a2e7404ce79b367567d8daba'), 'transactionIndex': 0, 'blockHash': HexBytes('0xac010c1c4637348ef0d121ad912245993374c1df1d9ee4cfd5add560e35cc7ce'), 'blockNumber': 152, 'from': '0xa508dD875f10C33C52a8abb20E16fc68E981F186', 'to': '0x1bf51d06070d22b2B66F06DAeeB96A8708D5090A', 'gasUsed': 1405120, 'cumulativeGasUsed': 1405120, 'contractAddress': None, 'logs': [], 'status': 1, 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [7]:
projects = contract.functions.get_all_projects_name().call()

for idx,p in enumerate(projects):
    print("%d. %s" % (idx+1,p))

1. project1


In [8]:
# cid = IPFS_API.Publish('./test.txt')
# print(cid)
# print(IPFS_API.CatFile(cid).decode('UTF-8', 'ignore'))